# PostgreSQL in Python using SQLAlchemy

## The Old Way of Writing Database Code in Python
We're going to use the library `sqlite3` to create a simple database with two tables **Person** and **Address** in the following design:

![db_design](https://www.pythoncentral.io/wp-content/uploads/2013/04/SQLAlchemyPersonAddress.png)

To create tables and insert data into tables, type:

In [1]:
import sqlite3
conn = sqlite3.connect('example.db')
 
c = conn.cursor()
c.execute('''
          CREATE TABLE person
          (id INTEGER PRIMARY KEY ASC, name varchar(250) NOT NULL)
          ''')
c.execute('''
          CREATE TABLE address
          (id INTEGER PRIMARY KEY ASC, street_name varchar(250), street_number varchar(250),
           post_code varchar(250) NOT NULL, person_id INTEGER NOT NULL,
           FOREIGN KEY(person_id) REFERENCES person(id))
          ''')
 
c.execute('''
          INSERT INTO person VALUES(1, 'Sarun Gulyanon')
          ''')
c.execute('''
          INSERT INTO address VALUES(1, 'Newington Road', '15', '12121', 1)
          ''')
 
conn.commit()
conn.close()

To retrieve data from databases, type:

In [2]:
conn = sqlite3.connect('example.db')
c = conn.cursor()
c.execute('SELECT * FROM person')
print(c.fetchall())
c.execute('SELECT * FROM address')
print(c.fetchall())

[(1, 'Sarun Gulyanon')]
[(1, 'Newington Road', '15', '12121', 1)]


In [3]:
c.execute('DROP TABLE person')
c.execute('DROP TABLE address')
conn.close()

----

## Python's SQLAlchemy and Declarative
There are three most important components in writing SQLAlchemy code:

* A Table that represents a table in a database.
* A mapper that maps a Python class to a table in a database.
* A class object that defines how a database record maps to a normal Python object.

Instead of having to write code for Table, mapper and the class object at different places, SQLAlchemy's declarative allows a Table, a mapper and a class object to be defined at once in one class definition.

The following declarative definitions specify the same tables:

In [4]:
!pip install sqlalchemy

In [5]:
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine
 
Base = declarative_base()
 
class Person(Base):
    __tablename__ = 'person'
    # Here we define columns for the table person
    # Notice that each column is also a normal Python instance attribute.
    id = Column(Integer, primary_key=True)
    name = Column(String(250), nullable=False)
    
class Address(Base):
    __tablename__ = 'address'
    # Here we define columns for the table address.
    # Notice that each column is also a normal Python instance attribute.
    id = Column(Integer, primary_key=True)
    street_name = Column(String(250))
    street_number = Column(String(250))
    post_code = Column(String(250), nullable=False)
    person_id = Column(Integer, ForeignKey('person.id'))
    person = relationship(Person)

# Create an engine that stores data in the local directory's
# sqlalchemy_example.db file.
engine = create_engine('sqlite:///sqlalchemy_example.db')
 
# Create all tables in the engine. This is equivalent to "Create Table"
# statements in raw SQL.
Base.metadata.create_all(engine)


Add records into the database.

In [6]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
 
engine = create_engine('sqlite:///sqlalchemy_example.db')
# Bind the engine to the metadata of the Base class so that the
# declaratives can be accessed through a DBSession instance
Base.metadata.bind = engine
 
DBSession = sessionmaker(bind=engine)
# A DBSession() instance establishes all conversations with the database
# and represents a "staging zone" for all the objects loaded into the
# database session object. Any change made against the objects in the
# session won't be persisted into the database until you call
# session.commit(). If you're not happy about the changes, you can
# revert all of them back to the last commit by calling
# session.rollback()
session = DBSession()
 
# Insert a Person in the person table
new_person = Person(name='John Doe')
session.add(new_person)
session.commit()
 
# Insert an Address in the address table
new_address = Address(street_name='North Street', street_number='21', 
                      post_code='46202', person=new_person)
session.add(new_address)
session.commit()

To query, type:

In [7]:
engine = create_engine('sqlite:///sqlalchemy_example.db')
Base.metadata.bind = engine

DBSession = sessionmaker()
DBSession.bind = engine
session = DBSession()
# Make a query to find all Persons in the database
print(session.query(Person).all())

# Return the first Person from all Persons in the database
person = session.query(Person).first()
print(person.name)

# Find all Address whose person field is pointing to the person object
print(session.query(Address).filter(Address.person == person).all())

# Retrieve one Address whose person field is point to the person object
print(session.query(Address).filter(Address.person == person).one())

address = session.query(Address).filter(Address.person == person).one()
print(address.id, address.street_name, address.street_number, 
      address.post_code, address.person_id)

[<__main__.Person object at 0x7f02e7c37160>, <__main__.Person object at 0x7f02e7c371d0>]
John Doe
1 North Street 21 46202 1
